In [1]:
import numpy as np
import bokeh as bk
import pandas as pd
from pyproj import Proj, transform

In [2]:
accidents = pd.read_pickle(r"C:\Users\Max\Documents\OSM_BootCampExercises\Computation\Wk2_Visualization\fars_data\final_accidents2.pickle")

In [3]:
from_proj = Proj(init="epsg:4326")
to_proj = Proj(init="epsg:3857")

def convert(longitudes, latitudes):
    """Converts latlon coordinates to meters.
    Inputs:
    longitudes (array-like) : array of longitudes
    latitudes (array-like) : array of latitudes
    Example:
    x,y = convert(accidents.LONGITUD, accidents.LATITUDE)
    """
    x_vals = []
    y_vals = []
    for lon, lat in zip(longitudes, latitudes):
        x, y = transform(from_proj, to_proj, lon, lat)
        x_vals.append(x)
        y_vals.append(y)
    
    return x_vals, y_vals

accidents["x"], accidents["y"] = convert(accidents.LONGITUD, accidents.LATITUDE)

In [4]:
from bokeh.plotting import figure, output_file, show

In [5]:
from bokeh.models import ColumnDataSource

In [6]:
from bokeh.plotting import Figure
from bokeh.models import WMTSTileSource

fig = Figure(plot_width=1100, plot_height=650, webgl=True,
    x_range=(-13000000, -7000000), y_range=(2750000, 6250000),
    tools=["wheel_zoom", "pan"], active_scroll="wheel_zoom")
fig.axis.visible = False

STAMEN_TONER_BACKGROUND = WMTSTileSource(
    url='http://tile.stamen.com/toner-background/{Z}/{X}/{Y}.png',
    attribution=('Map tiles by <a href="http://stamen.com">Stamen Design</a> '
                 'under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>.'
                 'Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, '
                 'under <a href="http://www.openstreetmap.org/copyright">ODbL</a>'
                )
)

fig.add_tile(STAMEN_TONER_BACKGROUND);

In [7]:
drunk = ColumnDataSource(accidents[accidents.DRUNK_DR ==1][['x', 'y']])
speed = ColumnDataSource(accidents[accidents.SP ==1][['x', 'y']])
other = ColumnDataSource(accidents[(accidents.DRUNK_DR == 0) & (accidents.SP == 0)][['x', 'y']])

In [10]:
#fig = figure(plot_width=500, plot_height=500, webgl=True)
cir_drunk = fig.circle(x='x', y='y', source=drunk, size=2, 
            fill_color='blue', line_width=0, fill_alpha=1)
cir_speed = fig.circle(x='x', y='y', source=speed, size=2, 
            fill_color='red', line_width=0,  fill_alpha=.5)
cir_other = fig.circle(x='x', y='y', source=other, size=1.2, 
            fill_color='green',  line_width=0, fill_alpha=.1)
show(fig)